In [1]:
!pip install pyspark==3.1.2 -q
!pip install findspark -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 MB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 8.9 MB/s eta 0:00:00


In [2]:
import findspark
findspark.init()

from pyspark.sql import SparkSession
from pyspark.sql.functions import rand
#import tokenizer
from pyspark.ml.feature import Tokenizer

In [4]:
spark = SparkSession.builder.appName("Exercises - Feature Extraction and Transformation using Spark").getOrCreate()

In [5]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/proverbs.csv


--2024-05-14 10:12:13--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/proverbs.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 846 [text/csv]
Saving to: ‘proverbs.csv’

proverbs.csv        100%[===================>]     846  --.-KB/s    in 0s      

2024-05-14 10:12:14 (153 MB/s) - ‘proverbs.csv’ saved [846/846]



In [6]:
# Load proverbs dataset
textdata = spark.read.csv("proverbs.csv", header=True, inferSchema=True)

In [7]:
textdata.show(truncate = False)

+---+-----------------------------------------------------------+
|id |text                                                       |
+---+-----------------------------------------------------------+
|1  |When in Rome do as the Romans do.                          |
|2  |Do not judge a book by its cover.                          |
|3  |Actions speak louder than words.                           |
|4  |A picture is worth a thousand words.                       |
|5  |If at first you do not succeed try try again.              |
|6  |Practice makes perfect.                                    |
|7  |An apple a day keeps the doctor away.                      |
|8  |When the going gets tough the tough get going.             |
|9  |All is fair in love and war.                               |
|10 |Too many cooks spoil the broth.                            |
|11 |You can not make an omelette without breaking eggs.        |
|12 |The early bird catches the worm.                           |
|13 |Bette

In [8]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/mpg.csv


--2024-05-14 10:12:55--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/mpg.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13891 (14K) [text/csv]
Saving to: ‘mpg.csv’

mpg.csv             100%[===================>]  13.57K  --.-KB/s    in 0s      

2024-05-14 10:12:56 (255 MB/s) - ‘mpg.csv’ saved [13891/13891]



In [9]:
mpgdata = spark.read.csv("mpg.csv", header=True, inferSchema=True)
mpgdata.show()

+----+---------+-----------+----------+------+----------+----+--------+
| MPG|Cylinders|Engine Disp|Horsepower|Weight|Accelerate|Year|  Origin|
+----+---------+-----------+----------+------+----------+----+--------+
|15.0|        8|      390.0|       190|  3850|       8.5|  70|American|
|21.0|        6|      199.0|        90|  2648|      15.0|  70|American|
|18.0|        6|      199.0|        97|  2774|      15.5|  70|American|
|16.0|        8|      304.0|       150|  3433|      12.0|  70|American|
|14.0|        8|      455.0|       225|  3086|      10.0|  70|American|
|15.0|        8|      350.0|       165|  3693|      11.5|  70|American|
|18.0|        8|      307.0|       130|  3504|      12.0|  70|American|
|14.0|        8|      454.0|       220|  4354|       9.0|  70|American|
|15.0|        8|      400.0|       150|  3761|       9.5|  70|American|
|10.0|        8|      307.0|       200|  4376|      15.0|  70|American|
|15.0|        8|      383.0|       170|  3563|      10.0|  70|Am

# Task 1 - Tokenizer


In [10]:
textdata.show(truncate = False)

+---+-----------------------------------------------------------+
|id |text                                                       |
+---+-----------------------------------------------------------+
|1  |When in Rome do as the Romans do.                          |
|2  |Do not judge a book by its cover.                          |
|3  |Actions speak louder than words.                           |
|4  |A picture is worth a thousand words.                       |
|5  |If at first you do not succeed try try again.              |
|6  |Practice makes perfect.                                    |
|7  |An apple a day keeps the doctor away.                      |
|8  |When the going gets tough the tough get going.             |
|9  |All is fair in love and war.                               |
|10 |Too many cooks spoil the broth.                            |
|11 |You can not make an omelette without breaking eggs.        |
|12 |The early bird catches the worm.                           |
|13 |Bette

In [13]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="text", outputCol="words")
textdata = tokenizer.transform(textdata)

In [14]:
#display the tokenized data
textdata.select("id","words").show(truncate=False)

+---+------------------------------------------------------------------------+
|id |words                                                                   |
+---+------------------------------------------------------------------------+
|1  |[when, in, rome, do, as, the, romans, do.]                              |
|2  |[do, not, judge, a, book, by, its, cover.]                              |
|3  |[actions, speak, louder, than, words.]                                  |
|4  |[a, picture, is, worth, a, thousand, words.]                            |
|5  |[if, at, first, you, do, not, succeed, try, try, again.]                |
|6  |[practice, makes, perfect.]                                             |
|7  |[an, apple, a, day, keeps, the, doctor, away.]                          |
|8  |[when, the, going, gets, tough, the, tough, get, going.]                |
|9  |[all, is, fair, in, love, and, war.]                                    |
|10 |[too, many, cooks, spoil, the, broth.]         

# 2 - CountVectorizer
CountVectorize the column "words" of the "textdata" dataframe and store the result in the column "features"

In [15]:
from pyspark.ml.feature import CountVectorizer

cv = CountVectorizer(inputCol = 'words', outputCol="features")

model = cv.fit(textdata)
textdata = model.transform(textdata)

In [16]:
# Show the resulting dataframe
textdata.select("words","features").show(truncate=False)

+------------------------------------------------------------------------+----------------------------------------------------------------------------+
|words                                                                   |features                                                                    |
+------------------------------------------------------------------------+----------------------------------------------------------------------------+
|[when, in, rome, do, as, the, romans, do.]                              |(99,[0,4,5,6,17,38,78,95],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])                |
|[do, not, judge, a, book, by, its, cover.]                              |(99,[1,3,4,19,21,22,62,70],[1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])               |
|[actions, speak, louder, than, words.]                                  |(99,[9,16,58,66,89],[1.0,1.0,1.0,1.0,1.0])                                  |
|[a, picture, is, worth, a, thousand, words.]                            |(99,[1,2,16,61

# 3 - StringIndexer
Convert the string column "Origin" to a numeric column "OriginIndex" in the dataframe "mpgdata"

In [17]:
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="Origin", outputCol="OriginIndex")
indexed = indexer.fit(mpgdata).transform(mpgdata)

In [19]:
#show the dataframe
indexed.orderBy(rand()).show()

+----+---------+-----------+----------+------+----------+----+--------+-----------+
| MPG|Cylinders|Engine Disp|Horsepower|Weight|Accelerate|Year|  Origin|OriginIndex|
+----+---------+-----------+----------+------+----------+----+--------+-----------+
|18.0|        6|      250.0|        88|  3021|      16.5|  73|American|        0.0|
|11.0|        8|      350.0|       180|  3664|      11.0|  73|American|        0.0|
|25.5|        4|      140.0|        89|  2755|      15.8|  77|American|        0.0|
|33.5|        4|       85.0|        70|  1945|      16.8|  77|Japanese|        1.0|
|15.0|        8|      429.0|       198|  4341|      10.0|  70|American|        0.0|
|28.0|        4|      112.0|        88|  2605|      19.6|  82|American|        0.0|
|20.0|        6|      198.0|        95|  3102|      16.5|  74|American|        0.0|
|28.0|        4|       97.0|        92|  2288|      17.0|  72|Japanese|        1.0|
|23.7|        3|       70.0|       100|  2420|      12.5|  80|Japanese|     

# Exercise 4 - StandardScaler
Create a single column named "feaures" using the columns "Cylinders", "Engine Disp", "Horsepower", "Weight"

In [20]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=["Cylinders", "Engine Disp", "Horsepower", "Weight"], outputCol="features")

mpg_transformed_data = assembler.transform(mpgdata)

#show the dataframe
mpg_transformed_data.select("MPG","features").show(truncate = False)

+----+------------------------+
|MPG |features                |
+----+------------------------+
|15.0|[8.0,390.0,190.0,3850.0]|
|21.0|[6.0,199.0,90.0,2648.0] |
|18.0|[6.0,199.0,97.0,2774.0] |
|16.0|[8.0,304.0,150.0,3433.0]|
|14.0|[8.0,455.0,225.0,3086.0]|
|15.0|[8.0,350.0,165.0,3693.0]|
|18.0|[8.0,307.0,130.0,3504.0]|
|14.0|[8.0,454.0,220.0,4354.0]|
|15.0|[8.0,400.0,150.0,3761.0]|
|10.0|[8.0,307.0,200.0,4376.0]|
|15.0|[8.0,383.0,170.0,3563.0]|
|11.0|[8.0,318.0,210.0,4382.0]|
|10.0|[8.0,360.0,215.0,4615.0]|
|15.0|[8.0,429.0,198.0,4341.0]|
|21.0|[6.0,200.0,85.0,2587.0] |
|17.0|[8.0,302.0,140.0,3449.0]|
|9.0 |[8.0,304.0,193.0,4732.0]|
|14.0|[8.0,340.0,160.0,3609.0]|
|22.0|[6.0,198.0,95.0,2833.0] |
|14.0|[8.0,440.0,215.0,4312.0]|
+----+------------------------+
only showing top 20 rows



Use StandardScaler to scale the "features" column of the dataframe "mpg_transformed_data" and save the scaled data into the "scaledFeatures" column.

In [21]:
from pyspark.ml.feature import StandardScaler

scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=True)

scalerModel = scaler.fit(mpg_transformed_data)

scaledData = scalerModel.transform(mpg_transformed_data)

In [22]:
# Show the scaled data
scaledData.select("features","scaledFeatures").show(truncate = False)

+------------------------+-----------------------------------------------------------------------------------+
|features                |scaledFeatures                                                                     |
+------------------------+-----------------------------------------------------------------------------------+
|[8.0,390.0,190.0,3850.0]|[1.48205302652896,1.869079955831451,2.222084561602166,1.027093462353608]           |
|[6.0,199.0,90.0,2648.0] |[0.3095711165403583,0.043843985634147174,-0.37591456792553746,-0.38801882543985255]|
|[6.0,199.0,97.0,2774.0] |[0.3095711165403583,0.043843985634147174,-0.1940546288585982,-0.2396792678175763]  |
|[8.0,304.0,150.0,3433.0]|[1.48205302652896,1.0472459587792617,1.1828849097910845,0.5361601645084557]        |
|[8.0,455.0,225.0,3086.0]|[1.48205302652896,2.4902335582546176,3.131384256936862,0.12763773200901246]        |
|[8.0,350.0,165.0,3693.0]|[1.48205302652896,1.4868315851095026,1.57258477922024,0.8422576643639463]          |
|

# Stop Spark Session


In [23]:
spark.stop()